In [1]:
import numpy as np
import pandas as pd
import random
import math

In [2]:
df = pd.read_csv("../spambase/spambase.data", names = word_labels, header=None) 
df.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_freq1,char_freq2,char_freq3,char_freq4,char_freq5,char_freq6,cap_run_length_avg,cap_run_length_longest,cap_run_length_total,label
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
df['label'].replace(0, -1,inplace=True)
df.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_freq1,char_freq2,char_freq3,char_freq4,char_freq5,char_freq6,cap_run_length_avg,cap_run_length_longest,cap_run_length_total,label
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [4]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    index_list = df.index.tolist()
    test_indexes = random.sample(population=index_list, k=test_size)

    test_df = df.loc[test_indexes]
    train_df = df.drop(test_indexes)
    
    return train_df, test_df

In [5]:
random.seed(3)
train_df, test_df = train_test_split(df, 0.20)

In [6]:
X_train = train_df.iloc[: , :-1]
y_train = train_df.iloc[: , -1]

X_test = test_df.iloc[: , :-1]
y_test = test_df.iloc[: , -1]

In [7]:
def make_clf_dict():
    clf_dict = {'label': 1, 'feature_name': None, 'threshold': None, 'alpha': None}
    return clf_dict

In [8]:
def predict(X):
    n_samples = np.shape(X)[0]
    y_pred = np.zeros((n_samples, 1))
    for clf in classifiers:
        predictions = np.ones(np.shape(y_pred))
        index_neg = (clf['label'] * X[clf['feature_name']] < clf['label'] * clf['threshold'])
        predictions[index_neg] = -1
        y_pred += clf['alpha'] * predictions
    y_pred = np.sign(y_pred).flatten()

    return y_pred




In [9]:
iters = 1000

def adaboost(X, y):
    n_samples, n_features = np.shape(X)
    w = np.full(n_samples, (1 / n_samples))
    classifiers = []
    for i in range(iters):
        clf = make_clf_dict()
        min_err = float('inf')
        for column in X_train.columns:
            feature_values = np.expand_dims(X[column], axis=1)
            unique_values = np.unique(feature_values)
 
            for threshold in unique_values:
                p = 1
    
                prediction = np.ones(np.shape(y))
             
                prediction[X[column] < threshold] = -1

                error = sum(w[y != prediction])

                if error > 0.5:
                    error = 1 - error
                    p = -1

                if error < min_err:
                    clf['label'] = p
                    clf['threshold'] = threshold
                    clf['feature_name'] = column
                    min_err = error

        clf['alpha'] = 0.5 * math.log((1.0 - min_err) / (min_err + 1e-10))

        predictions = np.ones(np.shape(y))

        index_neg = (clf['label'] * X[clf['feature_name']] < clf['label'] * clf['threshold'])

        predictions[index_neg] = -1

        w *= np.exp(-clf['alpha'] * y * predictions)

        w /= np.sum(w)

        classifiers.append(clf)
    return classifiers



In [10]:
classifiers = adaboost(X_train, y_train)
y_pred = predict(X_test)




In [11]:
classifiers

[{'label': 1,
  'feature_name': 'char_freq4',
  'threshold': 0.084,
  'alpha': 0.6882377113777169},
 {'label': 1,
  'feature_name': 'char_freq5',
  'threshold': 0.019,
  'alpha': 0.6228088061977658},
 {'label': -1,
  'feature_name': 'hp',
  'threshold': 0.1,
  'alpha': 0.4269492682024884},
 {'label': 1,
  'feature_name': 'remove',
  'threshold': 0.02,
  'alpha': 0.4851532637554392},
 {'label': 1,
  'feature_name': 'cap_run_length_longest',
  'threshold': 10,
  'alpha': 0.36781909063787244},
 {'label': 1,
  'feature_name': 'your',
  'threshold': 0.62,
  'alpha': 0.2619125490356106},
 {'label': -1,
  'feature_name': 'george',
  'threshold': 0.01,
  'alpha': 0.2526422307316929},
 {'label': 1,
  'feature_name': 'free',
  'threshold': 1.08,
  'alpha': 0.27059219164757936},
 {'label': -1,
  'feature_name': 're',
  'threshold': 0.13,
  'alpha': 0.2375124945508494},
 {'label': -1,
  'feature_name': 'char_freq4',
  'threshold': 0.037000000000000005,
  'alpha': 0.1914944565533873},
 {'label': 1,

In [12]:
np.mean(np.array(y_test) == y_pred)

0.9347826086956522

In [14]:
# testPredict = pd.DataFrame()
# trainPredict = pd.DataFrame()
# testPredict['y_predict'] = y_pred
# trainPredict['y_predict'] = y_train_pred


# testPredict.to_csv('y_test_predict.csv')
# trainPredict.to_csv('y_train_predict.csv')

In [15]:
len(classifiers)

1000